#### Imports

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)


C:\Users\rober\AppData\Local\Temp\ipykernel_34432\421210305.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from sentence_transformers import SentenceTransformer, util

c:\Users\rober\OneDrive\PyCharmProjects\cuda_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch import device, cuda, save, load

In [4]:
device = device('cuda' if cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
model = SentenceTransformer('all-mpnet-base-v2')
model.to(device)

modules.json: 100%|██████████| 349/349 [00:00<00:00, 349kB/s]
c:\Users\rober\OneDrive\PyCharmProjects\cuda_env\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rober\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config_sentenc

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
)

In [ ]:
dataset_df = pd.read_csv('../fine-tuning-experimentation/data/train-test-split-data/eclipse_val.csv', index_col='bug_id')

In [ ]:
dataset_df.head()

#### Encode the dataset

In [ ]:
encoded_desc = []
report_id_list = []
num = 0
for index, row in dataset_df.iterrows():
    try:
        encoded = model.encode(row[5])
    except:
        encoded = None
    print(num)
    report_id = index
    encoded_desc.append(encoded)
    report_id_list.append(report_id)
    num = num + 1

dataset_encoded_df = pd.DataFrame()
dataset_encoded_df["encoded_desc"] = encoded_desc
dataset_encoded_df["bug_id"] = report_id_list

In [ ]:
dataset_encoded_df = dataset_encoded_df.set_index('bug_id')
dataset_encoded_df.head()

#### Convert encoded dataset to dict and save to pytorch file

In [ ]:
enc_dict = dataset_encoded_df.to_dict()

In [ ]:
path = './encoding_data/mpnet_eclipse_val_enc.pt'
save(enc_dict, path)

In [ ]:
loaded_enc_dict = load('./encoding_data/mpnet_eclipse_val_enc.pt')

In [ ]:
load_enc1 = loaded_enc_dict['encoded_desc'][278431]
load_enc2 = loaded_enc_dict['encoded_desc'][278431]
cossim = util.cos_sim(load_enc1, load_enc2)
print(cossim)